In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ast

# 1. Data Loading and Preprocessing

def load_and_preprocess_data(file_path):
    # Load the CSV file without headers
    df = pd.read_csv(file_path, header=None)

    # Convert the string representation of lists to actual lists
    df[1] = df[1].apply(ast.literal_eval)

    # Ensure all input arrays are of size 5900
    desired_length = 5900
    df[1] = df[1].apply(lambda x: x[:desired_length] if len(x) > desired_length else x + [0] * (desired_length - len(x)))

    # Encode the labels to numerical values
    label_encoder = LabelEncoder()
    df[0] = label_encoder.fit_transform(df[0])

    # Prepare inputs (X) and outputs (y)
    X = torch.tensor(df[1].tolist(), dtype=torch.float32)
    y = torch.tensor(df[0].values, dtype=torch.long)

    return X, y, label_encoder

# Load data
file_path = 'F:/n/combined_data.csv'
X, y, label_encoder = load_and_preprocess_data(file_path)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 2. Neural Network Design

class ANNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ANNClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Model parameters
input_size = 5900
hidden_size = 128
output_size = len(label_encoder.classes_)

# Instantiate the model
model = ANNClassifier(input_size, hidden_size, output_size)

# 3. Training

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# 4. Evaluation

# Evaluate on test set
with torch.no_grad():
    y_pred = model(X_test)
    _, predicted = torch.max(y_pred, 1)
    accuracy = accuracy_score(y_test, predicted)
    print(f'Accuracy on test data: {accuracy * 100:.2f}%')

# 5. Prediction Function

def classify_input(input_data):
    input_tensor = torch.tensor(input_data, dtype=torch.float32)
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted_class = torch.max(output, 0)
    return label_encoder.inverse_transform([predicted_class.item()])[0]

# Example prediction
example_input = [0] * 5900  # Replace with a real example
predicted_label = classify_input(example_input)
print(f'Predicted Label: {predicted_label}')


Epoch [5/20], Loss: 1.8946
Epoch [10/20], Loss: 1.8369
Epoch [15/20], Loss: 1.7616
Epoch [20/20], Loss: 1.6649
Accuracy on test data: 78.95%
Predicted Label: Car
